In [3]:
import os

import torch
from tqdm.notebook import tqdm


device = 'cuda' if torch.cuda.is_available() else 'cpu'

os.cpu_count(), torch.cuda.get_device_name() if device == "cuda" else None

(12, 'GeForce RTX 2060')

In [ ]:
! wget -nc https://storage.googleapis.com/doctttttquery_git/t5-base.zip
! unzip -o t5-base.zip

In [64]:
from transformers import T5Config, T5Tokenizer, T5ForConditionalGeneration

# Instantiate and load the QG model to the GPU. 
qg_tokenizer = T5Tokenizer.from_pretrained('t5-base')
qg_config = T5Config.from_pretrained('t5-base')
qg_model = T5ForConditionalGeneration.from_pretrained('model.ckpt-1004000', from_tf=True, config=qg_config)

qg_model.to(device)

True

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 5.80 GiB total capacity; 2.28 GiB already allocated; 8.56 MiB free; 2.28 GiB reserved in total by PyTorch)

In [54]:
def generate_questions_doc_t5_query(context, q=1):
    """
    Generate q questions for the provided text context.
    """
    
    # Append an end of sequence token </s> after the context.
    doc_text = f"{context} </s>"

    with torch.no_grad():
        # TODO: We can probably batch this.
        input_ids = qg_tokenizer.encode(doc_text, return_tensors='pt').to(device)
        outputs = qg_model.generate(
            input_ids=input_ids,
            max_length=64,
            do_sample=False,
            num_return_sequences=q,
        )

    return [qg_tokenizer.decode(output) for output in outputs]

In [65]:
# Extract from the Wikipedia article on Language Models.
# https://en.wikipedia.org/wiki/Language_model

corpus = """
    A statistical language model is a probability distribution over sequences of words. 
    Given such a sequence, say of length m, it assigns a probability P ( w 1 , … , w m ) 
    {\displaystyle P(w_{1},\ldots ,w_{m})} P(w_{1},\ldots ,w_{m}) to the whole sequence.

    The language model provides context to distinguish between words and phrases that sound 
    similar. For example, in American English, the phrases "recognize speech" and "wreck a 
    nice beach" sound similar, but mean different things.

    Data sparsity is a major problem in building language models. Most possible word sequences 
    are not observed in training. One solution is to make the assumption that the probability 
    of a word only depends on the previous n words. This is known as an n-gram model or unigram 
    model when n = 1. The unigram model is also known as the bag of words model.

    Estimating the relative likelihood of different phrases is useful in many natural language
    processing applications, especially those that generate text as an output. Language modeling 
    is used in speech recognition,[1] machine translation,[2] part-of-speech tagging, parsing,[2]
    Optical Character Recognition, handwriting recognition,[3] information retrieval and other applications.

    In speech recognition, sounds are matched with word sequences. Ambiguities are easier to resolve 
    when evidence from the language model is integrated with a pronunciation model and an acoustic model.

    Language models are used in information retrieval in the query likelihood model. There, a separate
    language model is associated with each document in a collection. Documents are ranked based on the 
    probability of the query Q in the document's language model M d {\displaystyle M_{d}} M_d: P ( Q ∣ M d ) 
    {\displaystyle P(Q\mid M_{d})} P(Q\mid M_{d}). Commonly, the unigram language model is used for this purpose. 
""".split("\n\n")


[
    generate_questions_doc_t5_query(s) 
    for s in corpus
]

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 5.80 GiB total capacity; 2.28 GiB already allocated; 4.56 MiB free; 2.29 GiB reserved in total by PyTorch)